In [2]:
from pvtrace.scene.scene import Scene
from pvtrace.scene.renderer import MeshcatRenderer
from pvtrace.scene.node import Node
from pvtrace.algorithm import photon_tracer
from pvtrace.geometry.sphere import Sphere
from pvtrace.material.dielectric import Dielectric
from pvtrace.light.ray import Ray
import numpy as np
import logging
logging.getLogger("pvtrace").setLevel(logging.CRITICAL)

DEBUG:trimesh:searching for scad in: /Users/daniel/.pyenv/versions/pvtrace-3.7.2/bin:/Users/daniel/.pyenv/versions/pvtrace-3.7.2/bin:/usr/local/Cellar/pyenv/1.2.9/libexec:/Users/daniel/.pyenv/shims:/Users/daniel/miniconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/usr/local/MacGPG2/bin:/opt/X11/bin:/Applications/OpenSCAD.app/Contents/MacOS
DEBUG:trimesh:searching for blender in: /Users/daniel/.pyenv/versions/pvtrace-3.7.2/bin:/Users/daniel/.pyenv/versions/pvtrace-3.7.2/bin:/usr/local/Cellar/pyenv/1.2.9/libexec:/Users/daniel/.pyenv/shims:/Users/daniel/miniconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/usr/local/MacGPG2/bin:/opt/X11/bin:/Applications/blender.app/Contents/MacOS:/Applications/Blender.app/Contents/MacOS:/Applications/Blender/blender.app/Contents/MacOS
DEBUG:shapely.geos:Found GEOS DLL: <CDLL '/Users/daniel/.pyenv/versions/3.7.2/envs/pvtrace-3.7.2/lib/python3.7/site-packages/shapely/.dylibs/libgeos_c.1.dylib', handle 

# Example

Let's make a scene. The world is a giant sphere filled with air (radius 10) and shape we will ray trace is a sphere with radius 1.

## A simple scene

In [3]:
world = Node(
    name="world (air)",
    geometry=Sphere(
        radius=10.0,
        material=Dielectric.air()
    )
)
sphere = Node(
    name="sphere (glass)",
    geometry=Sphere(
        radius=1.0,
        material=Dielectric.glass()
    ),
    parent=world
)
scene = Scene(world)

## Using the visualiser
Let's visualise this scene with the meshcat based renderer.

In [4]:
vis = MeshcatRenderer()
vis.render(scene)
vis.vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7009/static/


## Making rays
Throw some rays at the scene. A ray has a position, direction and wavelength.

In [5]:
ray = Ray(
    position=(-1.0, 0.0, 1.1),
    direction=(1.0, 0.0, 0.0),
    wavelength=555.0
)

## Tracing the scene

Let's follow the ray through the scene using the `follow` function of the `photon_tracer` module. Drag the visualisation to see the path of the ray.

In [6]:
np.random.seed(0)
steps = photon_tracer.follow(ray, scene)
path, decisions = zip(*steps)
vis.add_ray_path(path)

The photon tracer simulates the propagation of the ray through the scene as if it was a photon. Automatic Fresnel reflection and refraction occurs at boundaries between materials for different refractive index. As you will see later, if the material also has an absopative or emissive properties rays can be absorbed and re-emitted.

The follow method returns a list of steps. At each step something happened. A step is just a tuple containing a ray and a decision enum.

In [7]:
steps[0]

(Ray(position=(-1.00, 0.00, 1.10), direction=(1.00, 0.00, 0.00), wavelength=555.00, is_alive=True),
 <Decision.EMIT: 4>)

The first step is the creation of the ray with an `EMIT` decision.

In [8]:
steps[1]

(Ray(position=(9.94, 0.00, 1.10), direction=(1.00, 0.00, 0.00), wavelength=555.00, is_alive=True),
 <Decision.TRAVEL: 5>)

The ray then travels to an intersection point.

In [9]:
steps[2]

(Ray(position=(9.94, 0.00, 1.10), direction=(1.00, 0.00, 0.00), wavelength=555.00, is_alive=False),
 <Decision.KILL: 6>)

The ray then hits the world node and is killed.

Let's make a ray that intersects with the sphere and see what happens (you may need to scroll up to see the viewer).

In [10]:
ray = Ray(
    position=(-1.0, 0.0, 0.9),
    direction=(1.0, 0.0, 0.0),
    wavelength=650.0
)
steps = photon_tracer.follow(ray, scene)
path, decisions = zip(*steps)
vis.add_ray_path(path)

This is a Monte Carlo simulation, the ray will not necessarily take the same path every time.

Let's generate 100 identical rays and see what happens.

In [11]:
import time
for _ in range(100):
    steps = photon_tracer.follow(ray, scene)
    path, decisions = zip(*steps)
    vis.add_ray_path(path)
    time.sleep(0.001)  # allow the renderer a chance to redraw

The other tutorial and example files show how to photon trace materials which also absorb and emit light and how to import complex geometies from mesh files and ray trace those.